In [4]:
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.1+cpu.html
!pip install torch-geometric-temporal

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cpu.html
                                              0.0/500.0 kB ? eta -:--:--
     --                                    30.7/500.0 kB 660.6 kB/s eta 0:00:01
     ------                                81.9/500.0 kB 919.0 kB/s eta 0:00:01
     ---------------------                  276.5/500.0 kB 1.9 MB/s eta 0:00:01
     ----------------------------------     450.6/500.0 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 500.0/500.0 kB 2.1 MB/s eta 0:00:00
                                              0.0/1.2 MB ? eta -:--:--
                                              0.0/1.2 MB ? eta -:--:--
                                              0.0/1.2 MB ? eta -:--:--
                                              0.0/1.2 MB 65.2 kB/s eta 0:00:19
                                              0.0/1.2 MB 65.2 kB/s eta 0:00:19
                                              0.0/1.2 MB 65.2 kB/s eta 0:00:19
     -       

In [5]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
writer.add_scalar('loss', 1)

In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split


# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# graph deep learning
import torch_geometric
import torch_geometric_temporal
import networkx as nx

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch_geometric
%matplotlib inline

# For Progress Bar
from tqdm.auto import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

## get data

In [8]:
from Data.generate_data import generate_data, get_data, get_and_normalize_data

train_loader, valid_loader, test_loader, edge_index = generate_data(50)

## train config

In [ ]:
from torch_geometric_temporal.nn.attention import MTGNN

In [ ]:
model_args = {'gcn_true': True, 'buildA_true': False, 'gcn_depth': 2, 'num_nodes': 50, 'dropout': 0.3,
              'subgraph_size': 20, 'node_dim': 40, 'dilation_exponential': 2, 'conv_channels': 16,
              'residual_channels': 16, 'skip_channels': 32, 'end_channels': 64, 'seq_in_len': 168, 'in_dim': 7, 'seq_out_len': 1,
              'layers': 5, 'propalpha': 0.05, 'tanhalpha': 3, 'layer_norm_affline': True, 'kernel_size': 7,
              'kernel_set': [2, 3, 6, 7]
              }

In [ ]:
train_config = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'n_epochs': 30,     # Number of epochs.
    'learning_rate': 1e-3,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt',  # Your model will be saved here.
    'is_L1Loss': False
}

In [ ]:
model = MTGNN(model_args['gcn_true'], model_args['buildA_true'], model_args['gcn_depth'], model_args['num_nodes'], kernel_set=model_args['kernel_set'],
              kernel_size=model_args['kernel_size'],
        dropout=model_args['dropout'], subgraph_size=model_args['subgraph_size'],
        node_dim=model_args['node_dim'], dilation_exponential=model_args['dilation_exponential'],
        conv_channels=model_args['conv_channels'], residual_channels=model_args['residual_channels'],
        skip_channels=model_args['skip_channels'], end_channels=model_args['end_channels'],
        seq_length=model_args['seq_in_len'], in_dim=model_args['in_dim'], out_dim=model_args['seq_out_len'],
        layers=model_args['layers'], propalpha=model_args['propalpha'], tanhalpha=model_args['tanhalpha'], layer_norm_affline=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model =model.to(device)

## load model

In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/TopoMAD/models/model.ckpt'))

## train

In [ ]:
def trainer(train_config, train_loader, valid_loader, model, device, edge=None):
    criterion = nn.MSELoss(reduction='mean')  # TODO: use loss class

    if train_config['learning_rate']:  # TODO: use optim class
        optimizer = torch.optim.Adam(model.parameters(), lr=train_config['learning_rate'])
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=train_config['learning_rate'])

    writer = SummaryWriter()  # Writer of tensoboard.   # TODO: tensorboard bp

    if not os.path.isdir('./models'):
        os.mkdir('./models')  # Create directory of saving models.  # TODO: in init

    n_epochs, best_loss, early_stop_count = train_config['n_epochs'], math.inf, 0  # TODO: in init

    for epoch in range(n_epochs):  # TODO: update
        model.train()  # Set your model to train mode.
        train_loss_record = []  # set record to class

        train_rate_record = []  # set record to class

        for step, (x, y) in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()  # Set gradient to zero.
            x, y = x.to(device), y.to(device)  # Move your data to device.

            pred = model(x, edge)

            train_loss = criterion(pred, y)

            train_ace = torch.mean(torch.abs(pred - y) / torch.abs(y)) * 100

            train_loss.backward()  # Compute gradient(backpropagation).
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            optimizer.step()  # Update parameters.

            train_loss_record.append(train_loss.detach().item())  # set
            train_rate_record.append(train_ace.detach().item())

        mean_train_loss = sum(train_loss_record) / len(train_loss_record)
        mean_train_rate = sum(train_rate_record) / len(train_rate_record)

        writer.add_scalar('Loss/train', mean_train_loss, step)

        # Print the information. TODO: add to class
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {mean_train_loss:.5f},")
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] rate = {mean_train_rate:.5f},")

        model.eval()  # Set your model to evaluation mode.
        valid_loss_record = []
        valid_rate_record = []  # set record to class
        for x, y in tqdm(valid_loader):
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x, edge)

                loss = criterion(pred, y)
                valid_rate = torch.mean(torch.abs(pred - y) / torch.abs(y)) * 100

            valid_loss_record.append(loss.item())
            valid_rate_record.append(valid_rate.item())

        mean_valid_loss = sum(valid_loss_record) / len(valid_loss_record)
        mean_valid_rate = sum(valid_rate_record) / len(valid_rate_record)

        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {mean_valid_loss:.5f}")
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] rate = {mean_valid_rate:.5f}")

        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), train_config['save_path'])  # Save your best model
            print('Saving model with loss {:.3f}... -> best'.format(mean_valid_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= train_config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            print(f'\nbest loss is {best_loss}')
            return

In [ ]:
edge = edge.to(device)
trainer(train_config, train_loader, valid_loader, model, device, edge)

  0%|          | 0/315 [00:00<?, ?it/s]

[ Train | 001/030 ] loss = 227755.18085,
[ Train | 001/030 ] loss = 57.47195,


  0%|          | 0/105 [00:00<?, ?it/s]

[ Valid | 001/030 ] loss = 189238.75409
[ Valid | 001/030 ] rate = 47.50209
Saving model with loss 189238.754... -> best


  0%|          | 0/315 [00:00<?, ?it/s]

...

[ Valid | 029/030 ] loss = 173733.22444
[ Valid | 029/030 ] rate = 46.51993


  0%|          | 0/315 [00:00<?, ?it/s]

[ Train | 030/030 ] loss = 188095.11231,
[ Train | 030/030 ] loss = 53.22455,


  0%|          | 0/105 [00:00<?, ?it/s]

[ Valid | 030/030 ] loss = 171734.99412
[ Valid | 030/030 ] rate = 46.25861


In [ ]:
criterion = nn.L1Loss(reduction='mean').to(train_config['device'])
def predict(test_loader, model, criterion, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x, y in tqdm(test_loader):
        x, y = x.to(train_config['device']), y.to(train_config['device'])

        with torch.no_grad():
            pred = model(x)
            print(y)
            loss_rate = criterion(pred, y)
            print(loss_rate)
            preds.append(loss_rate)

    mean_pred_loss = sum(preds) / len(preds)
    print(f'loss: {mean_pred_loss:.4f}')
    return

predict(test_loader, model, criterion, device)

  0%|          | 0/108 [00:00<?, ?it/s]

AssertionError: ignored

In [ ]:
for x, y in train_loader:
  print(x.shape)

IndexError: ignored

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

In [ ]:
## edge index

In [ ]:
a = edge_index[0]
b = edge_index[1]
edges = []
for edge in zip(a, b):
  edges.append(edge)

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_edges_from(edges)
nx.write_adjlist(G, 'DatasetUpdate/edge.adjlist')

In [ ]:
G = nx.read_adjlist('DatasetUpdate/edge.adjlist')

In [ ]:
G.edges